#Installing Dependencies

In [ ]:
!pip install --upgrade langchain chromadb langchain-google-genai pypdf tiktoken langchain-community sentence-transformers langsmith


In [ ]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import TokenTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from google.colab import files
import warnings
warnings.filterwarnings("ignore")

Taking (Master Direction – Reserve Bank of India) input

In [ ]:
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

Saving Querry SET.pdf to Querry SET.pdf


In [ ]:
loader = PyPDFLoader(pdf_path)
documents = loader.load()

tokenizing the pdf into small chunks of overlap 50 token  to maintain the continuity

In [ ]:
splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=50)
docs = splitter.split_documents(documents)

# Embedding by Hugging Face model for Semantic Searching

In [ ]:
from getpass import getpass
os.environ["GOOGLE_API_KEY"] =getpass("ENTER your API KEY")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

ENTER your API KEY··········


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

#CHROMADB AS VECTOR DATABASE

In [ ]:
vectorstore = Chroma.from_documents(docs, embeddings, collection_name="pdf_gemini")
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [ ]:

llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=0.8)


qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True)

In [ ]:
def ask_from_pdf(query, min_words=3):
    result = qa_chain({"query": query})
    answer = result["result"]

    # If answer is too short, assume not found
    if len(answer.split()) < min_words:
        return "I’m not sure the answer is in the PDF."
    return answer

In [ ]:
query = "What is the difference between banks and NBFCs"
answer = ask_from_pdf(query)
print("Answer:", answer)


Answer: Based on the context provided, one key difference between banks and NBFCs relates to how they handle bank deposits:

For an NBFC, investments in fixed deposits with banks are not considered "financial assets," and the interest income from these deposits is not treated as "income from financial assets." The provided text states that this is because these activities are not covered under the definition of a “financial Institution” in section 45-I(c) of the RBI Act, 1934.

The context further clarifies that NBFCs can use bank deposits for "temporary parking of idle funds" or to fulfill initial registration capital requirements, but it is not part of their core financial business.


EVAlUATION

In [ ]:
rbi_faqs = [
{"question": "What is a Non-Banking Financial Company (NBFC)?",

"answer" : "A Non-Banking Financial Company (NBFC) is a company registered under the Companies Act, 1956 or Companies Act, 2013, and engaged in the business of loans and advances, acquisition of shares/stocks/bonds/debentures/securities issued by Government or local authority or other marketable securities of a like nature, leasing, hire-purchase, etc., as their principal business, but does not include any institution whose principal business is that of agriculture activity, industrial activity, purchase or sale of any goods (other than securities) or providing any services and sale/purchase/construction of immovable property. A non-banking institution which is a company and has principal business of receiving deposits under any scheme or arrangement in one lump sum or in installments by way of contributions or in any other manner, is also a non-banking financial company (Residuary non-banking company)."
},
{
  "question": "What does conducting financial activity as “principal business” mean?",
  "answer": "Financial activity as principal business is when a company’s financial assets constitute more than 50 per cent of the total assets (netted off by intangible assets) and income from financial assets constitute more than 50 per cent of the gross income. A company which fulfils both these criteria needs to get registered as NBFC with the Reserve Bank. The term 'principal business' has not been defined in the Reserve Bank of India Act, 1934. Hence, the Reserve Bank has defined it vide Press Release 1998-99/1269 dated April 08, 1999 so as to ensure that only companies predominantly engaged in financial activity get registered with it and are regulated and supervised by it. Hence, if there are companies engaged in agricultural operations, industrial activity, purchase and sale of goods, providing services or purchase, sale or construction of immovable property as their principal business and are doing some financial business in a small way, they will not be regulated by the Reserve Bank. Interestingly, this test is popularly known as 50-50 test and is applied to determine whether or not a company is into financial business."
},
{
  "question": "NBFCs are doing functions similar to banks. What is the difference between banks and NBFCs?",
  "answer": "Banks and NBFCs are different entities subject to different statutory and regulatory requirements. However, NBFCs lend and make investments and hence these activities are akin to that of banks. The major differences between banks and NBFCs are given below:\n\ni. NBFCs cannot accept demand deposits;\n\nii. NBFCs do not form part of the payment and settlement system and cannot issue cheques drawn on itself;\n\niii. Deposit insurance facility of Deposit Insurance and Credit Guarantee Corporation (DICGC) is not available to depositors of deposit taking NBFCs."
},
{
  "question": "Is it necessary that every NBFC should be registered with the Reserve Bank?",
  "answer": "In terms of Section 45-IA of the RBI Act, 1934, no NBFC can commence or carry on business of a non-banking financial institution without a) obtaining a certificate of registration from the Reserve Bank and without having a Net Owned Funds (NOF) of ₹10 crore with effect from October 01, 2022 (NBFCs seeking registration shall have NOF of ₹10 crore ab initio, and existing NBFCs have timeline upto March 31, 2027 to attain NOF of ₹10 crore). However, in terms of the powers conferred upon the Reserve Bank, to obviate dual regulation, certain categories of NBFCs which are regulated by other regulators are exempted from the requirement of registration with the Reserve Bank viz., Alternative Investment Fund/ Merchant Banking companies/ Stock broking companies registered with SEBI; Insurance Company holding a valid Certificate of Registration issued by IRDA; Nidhi companies as notified under Section 620A of the Companies Act, 1956; Chit companies doing the business of chits as defined in clause (b) of Section 2 of the Chit Funds Act, 1982; Stock Exchange or a Mutual Benefit company, etc."
},
{
  "question": "What are the requirements for registration with the Reserve Bank?",
  "answer": "A ‘company’ desirous of commencing the business of non-banking financial institution as defined under Section 45 I(a) of the RBI Act, 1934 should comply with the following:\n\ni. It should be a company incorporated under Section 3 of the Companies Act, 1956 or corresponding Section under the Companies Act, 2013;\n\nii. It should have a minimum net owned fund of ₹10 crore. (The minimum net owned fund requirements for specialized NBFCs are NBFC-Infrastructure Finance Company (NBFC-IFC) – ₹300 crore; Infrastructure Debt Fund – NBFC (IDF-NBFC) – ₹300 crore; Mortgage Guarantee Company (MGC) – ₹100 crore; Housing Finance Company (HFC) – ₹20 crore, Standalone Primary Dealers (SPDs) which undertake only the core activities – ₹150 crore and SPDs which also undertake non-core activities – ₹250 crore; NBFC-AA – ₹2 crore; and NBFC-P2P – ₹2 crore)."
},
{
  "question": "What is the procedure for application to the Reserve Bank for Registration?",
  "answer": "The applicant company is required to apply online on https://pravaah.rbi.org.in and also submit a physical copy of the application along with the necessary documents as per the process prescribed by the Reserve Bank vide its Press Release 2015-2016/2935 dated June 17, 2016 to the Chief General Manager-in-Charge, Department of Regulation, Reserve Bank of India, Central Office, 2nd Floor, Main Office Building, Shahid Bhagat Singh Marg, Fort, Mumbai-400 001."
},
{
  "question": "What are the essential documents required to be submitted along with the application form to the Reserve Bank?",
  "answer": "The application form and an indicative checklist of the documents required to be submitted along with the application is available on Reserve Bank’s website under NBFC Forms."
},
{
  "question": "What is Scale Based Regulatory Framework or SBR Framework for NBFCs?",
  "answer": "Over the years, the NBFC sector had evolved considerably in terms of size, complexity, and interconnectedness within the financial sector and hence there was a need to align the regulatory framework for NBFCs keeping in view their changing risk profile. Accordingly, the Reserve Bank has implemented a Scale-Based Regulatory Framework or SBR Framework for regulation of NBFCs w.e.f. October 01, 2022. The SBR Framework which is based on the principle of proportionality takes into account various factors like size, activity, complexity, interconnectedness, etc., within the financial sector for categorising NBFCs into various layers. The degree of regulations increases as one moves from lower to higher layers. SBR Framework classifies NBFCs into four layers. NBFCs in the lowest layer shall be known as NBFC – Base Layer (NBFC-BL). NBFCs in middle layer and upper layer shall be known as NBFC – Middle Layer (NBFC-ML) and NBFC – Upper Layer (NBFC-UL) respectively and are considered to be systemically significant. The Top Layer is ideally expected to be empty and will be known as NBFC - Top Layer (NBFC-TL) which will be populated only if the Reserve Bank is of the opinion that there is a substantial increase in the potential systemic risk from specific NBFCs in the Upper Layer."
},
{
  "question": "Does the Reserve Bank regulate all financial companies?",
  "answer": "No, the Reserve Bank does not regulate all financial companies. Depending upon the nature of activities, the financial companies may fall under the regulatory purview of other Regulators like SEBI, IRDAI, Government, etc. To name a few, the Merchant Banking Companies/Alternative Investment Fund Company/stock-exchanges/stock brokers/sub-brokers are regulated by Securities and Exchange Board of India, and Insurance companies are regulated by Insurance Regulatory and Development Authority. Similarly, Chit Fund Companies are regulated by the respective State Governments and Nidhi Companies are regulated by Ministry of Corporate Affairs, Government of India. Companies that do financial business but are regulated by other regulators are given specific exemption by the Reserve Bank from its regulatory requirements for avoiding duality of regulation. The categories of NBFCs which are exempted from certain provisions of the RBI Act, 1934 are specified in the ‘Master Direction - Exemptions from the provisions of RBI Act, 1934 dated August 25, 2016."
},
{
  "question": "What are the different types/categories of NBFCs registered with the Reserve Bank?",
  "answer": "NBFCs are categorized (a) in terms of the type of liabilities into Deposit and Non-Deposit accepting NBFCs; (b) regulatory structure of NBFCs under Scale Based Regulation into NBFC-Base Layer, NBFC-Middle Layer, NBFC-Upper Layer, and NBFC-Top Layer (as detailed in FAQ no.8 above); and (c) by the kind of activity they conduct.\n\nBased on the type of activities they conduct, the different types of NBFCs are as follows:\n\nI. Investment and Credit Company (ICC): ICC means any company which is a financial institution carrying on as its principal business - asset finance, the providing of finance whether by making loans or advances or otherwise for any activity other than its own and the acquisition of securities; and is not any other category of NBFCs as defined by the Reserve Bank in any of its Master Directions.\n\nII. Housing Finance Company (HFC): HFC shall mean a company that fulfils the following conditions:\n(a) It is an NBFC whose financial assets, in the business of providing finance for housing, constitute at least 60% of its total assets (netted off by intangible assets). Housing finance for this purpose shall mean providing finance as stated at clauses (a) to (k) of Paragraph 4.1.16 of the Master Direction – Non-Banking Financial Company – Housing Finance Company (Reserve Bank) Directions, 2021.\n(b) Out of the total assets (netted off by intangible assets), not less than 50% should be by way of housing finance for individuals as stated at clauses (a) to (e) of Paragraph 4.1.16 of the aforementioned master directions for HFCs.\n\nIII. Infrastructure Finance Company (IFC): IFC is a non-banking finance company (a) which deploys at least 75 per cent of its total assets towards infrastructure lending.\n\nIV. Infrastructure Debt Fund (IDF-NBFC): IDF-NBFC means a non-deposit taking NBFC which is permitted to (a) refinance post commencement operations date (COD) infrastructure projects that have completed at least one year of satisfactory commercial operations; and (b) finance toll operate transfer (TOT) projects as the direct lender.\n\nV. Core Investment Company (CIC): CIC is a NBFC carrying on the business of acquisition of shares and securities which satisfies the following conditions:\n(a) It holds not less than 90% of its net assets in the form of investment in equity shares, preference shares, debt or loans in group companies;\n(b) Its investments in the equity shares (including instruments compulsorily convertible into equity shares within a period not exceeding 10 years from the date of issue) in group companies and units of Infrastructure Investment Trusts (InvITs) only as sponsor constitutes not less than 60% of its net assets;\n(c) Provided that the exposure of such CICs towards InvITs shall be limited to their holdings as sponsors and shall not, at any point in time, exceed the minimum holding of units and tenor prescribed in this regard by SEBI (Infrastructure Investment Trusts) Regulations, 2014, as amended from time to time. It does not trade in its investments in shares, debt or loans in group companies except through block sale for the purpose of dilution or disinvestment;\n(d) it does not carry on any other financial activity referred to in Section 45I(c) and 45I(f) of the RBI act, 1934 except (i) investment in bank deposits, money market instruments, government securities, bonds or debentures issued by group companies; (ii) granting of loans to group companies; and (iii) issuing of guarantees on behalf of group companies;\n(e) Its asset size is ₹100 crore or above; and\n(f) It accepts public funds.\n\nVI. Micro Finance Institution (NBFC-MFI): “NBFC-MFI” means a non-deposit taking NBFC which has a minimum of 75 percent of its total assets deployed towards “microfinance loans” as defined under Reserve Bank of India (Regulatory Framework for Microfinance Loans) Directions, 2022 as under:\n(a) A microfinance loan is defined as a collateral-free loan given to a household having annual household income up to ₹3,00,000. For this purpose, the household shall mean an individual family unit, i.e., husband, wife and their unmarried children.\n(b) All collateral-free loans, irrespective of end use and mode of application/ processing/ disbursal (either through physical or digital channels), provided to low-income households, i.e., households having annual income up to ₹3,00,000, shall be considered as microfinance loans.\n(c) To ensure collateral-free nature of the microfinance loan, the loan shall not be linked with a lien on the deposit account of the borrower.\n(d) The NBFCs shall have a board-approved policy to provide the flexibility of repayment periodicity on microfinance loans as per borrowers’ requirement.\n\nVII. Non-Banking Financial Company – Factors (NBFC-Factors): NBFC-Factor is a non-deposit taking NBFC engaged in the principal business of factoring. The financial assets in the factoring business should constitute at least 50 percent of its total assets and its income derived from factoring business should not be less than 50 percent of its gross income.\n\nVIII. Mortgage Guarantee Companies (MGC): MGC means a company registered as mortgage guarantee company which primarily transacts the business of providing mortgage guarantee i.e., a guarantee for the repayment of an outstanding housing loan and interest accrued thereon up to the guaranteed amount to a creditor institution, on the occurrence of a trigger event. A mortgage guarantee company shall be deemed to primarily transact the business of providing mortgage guarantee when at least 90% of the business turnover is mortgage guarantee business or at least 90% of the gross income is from mortgage guarantee business.\n\nIX. Standalone Primary Dealers (SPDs): SPDs are primarily NBFCs which have been granted authorisation to undertake the Primary Dealer activities in Government Securities. SPDs may undertake a set of core and non-core activities which are clearly defined. SPDs support G-Sec market, (both primary and secondary) through various obligations like participating in primary auctions, market making in G-Secs, predominance of investment in G-Secs, achieving minimum secondary market turnover ratio, etc.\n\nX. Non-Operative Financial Holding Company (NOFHC): NOFHC means a non-deposit taking NBFC referred to in the 'Guidelines for Licensing of New Banks in the Private Sector' dated February 22, 2013, issued by the Reserve Bank, which holds the shares of a banking company and the shares of all other financial services companies in its group, whether regulated by the Reserve Bank or by any other financial regulator, to the extent permissible under the applicable regulatory prescriptions.\n\nXI. NBFC – Account Aggregator (NBFC-AA): NBFC-AA means a non-banking financial company as notified under in sub-clause (iii) of clause (f) of section 45-I of the RBI Act, that undertakes the business of an account aggregator, for a fee or otherwise. The “business of an account aggregator” means the business of providing under a contract, the service of, retrieving or collecting such financial information pertaining to its customer, as may be specified by the Reserve Bank from time to time; and consolidating, organizing and presenting such information to the customer or any other financial information user as may be specified by the Bank; Provided that, the financial information pertaining to the customer shall not be the property of the Account Aggregator, and not be used in any other manner.\n\nXII. NBFC – Peer to Peer Lending Platform (NBFC-P2P): NBFC-P2P means a non-banking institution which carries on the business of a Peer to Peer Lending Platform i.e., acting as intermediary providing the services of loan facilitation via online medium or otherwise, to the participants of the platform."
},

  {
    "question": "What are the powers of the Reserve Bank with regard to 'Non-Bank Financial Companies’, that is, companies that meet the Principal Business Criteria or 50-50 criteria?",
    "answer": "The Reserve Bank has been empowered under the RBI Act 1934 to register, determine policy, issue directions, inspect, regulate, supervise and exercise surveillance over NBFCs that fulfil the principal business criteria or 50-50 criteria of principal business. The Reserve Bank can penalize NBFCs for violating the provisions of the RBI Act or the directions or orders issued by the Reserve Bank under RBI Act. The penal action may also include cancellation of the Certificate of Registration issued to the NBFC."
  },
  {
    "question": "What action can be taken against persons/financial companies making false claim of being regulated by the Reserve Bank?",
    "answer": "It is illegal for any person/ entity/ financial company to make a false claim of being regulated by the Reserve Bank to mislead the public to collect deposits and is liable for penal action under the Law. Information in this regard may be forwarded to the nearest office of the Reserve Bank and the Police."
  },
  {
    "question": "What action is taken if financial companies which are lending or making investments as their principal business do not obtain a Certificate of Registration from the Reserve Bank?",
    "answer": "If companies that are required to be registered with the Reserve Bank as NBFCs, are found to be conducting non-banking financial activity, such as, lending, investment or deposit acceptance as their principal business, without obtaining Certificate of Registration from the Reserve Bank, the same would be treated as contravention of the provisions of the RBI Act, 1934 and would invite penal action viz., penalty or fine or even prosecution in a Court of Law. If members of public come across any entity which undertakes non-banking financial activity but does not figure in the list of authorized NBFCs on the Reserve Bank’s website, they should inform the nearest Regional Office of the Reserve Bank, for appropriate action to be taken for contravention of the provisions of the RBI Act, 1934."
  },
  {
    "question": "Where can one find list of Registered NBFCs and instructions issued to NBFCs?",
    "answer": "The list of registered NBFCs is available on the web site of Reserve Bank (www.rbi.org.in) under ‘Regulation → Non-Banking’. Further, the instructions issued to NBFCs from time to time through circulars and/ or master directions are hosted on the Reserve Bank’s website under ‘Notifications’, and some instructions are issued through Official Gazette notifications and press releases as well."
  },
  {
    "question": "What are the regulations prescribed by the Reserve Bank for NBFCs?",
    "answer": "As part of regulatory framework prescribed by the Reserve Bank for NBFCs, the Reserve Bank prescribes prudential regulations viz., capital adequacy/ leverage, provisioning, corporate governance framework, etc.; conduct of business regulations viz., KYC/ AML regulations, fair practices code, etc.; and other miscellaneous regulations to ensure that NBFCs are financially sound and follow transparency in their operations. The regulations for NBFCs are contained in various master directions and notifications/ circulars issued from time to time, and are available on the website of the Reserve Bank (www.rbi.org.in) under ‘notifications’."
  },
  {
    "question": "What does the term public funds include? Is it the same as public deposits?",
    "answer": "Public funds are not the same as public deposits. Public funds include public deposits, inter-corporate deposits, bank finance and all funds received whether directly or indirectly from outside sources such as funds raised by issue of Commercial Papers, debentures etc. Even though public funds include public deposits in the general course, it may be noted that CICs as also non-deposit taking NBFCs are not allowed to accept public deposits. Further, indirect receipt of public funds means funds received not directly but through associates and group entities which have access to public funds."
  },
  {
    "question": "What are the various prudential regulations applicable to NBFCs?",
    "answer": "The Reserve Bank has issued detailed directions on prudential norms, vide Master Direction – Reserve Bank of India (Non-Banking Financial Company – Scale Based Regulation) 2023 (as amended from time to time). Applicable regulations vary based on the layer of the NBFC under Scale Based Regulatory Framework for NBFCs. Further, specialised categories of NBFCs viz., NBFC-P2P, NBFC-AA, CICs, SPDs, MGCs and HFCs shall be subject to respective master directions governing them. The directions, inter alia, prescribe guidelines on income recognition, asset classification and provisioning requirements applicable to NBFCs, exposure norms, disclosures in the balance sheet, requirement of capital adequacy, loan to value (LTV) ratio for NBFCs predominantly engaged in business of lending against gold jewellery, besides others."
  },
  {
    "question": "Please explain the terms ‘owned fund’ and ‘net owned fund’ in relation to NBFCs?",
    "answer": "Para 5.1.25 of Master Direction – Reserve Bank of India (Non-Banking Financial Company – Scale Based Regulation) 2023 (as amended from time to time) defines ‘Owned Fund’ as aggregate of the paid-up equity capital, preference shares which are compulsorily convertible into equity, free reserves, balance in share premium account and capital reserves representing surplus arising out of sale proceeds of asset, excluding reserves created by revaluation of asset; as reduced by accumulated balance of loss, book value of intangible assets and deferred revenue expenditure, if any. 'Net Owned Fund' is defined under Section 45-IA(7) of the RBI Act, 1934. As per this definition, the Net Owned Fund means– (a) aggregate of the paid-up equity capital and free reserves as disclosed in the latest balance-sheet of the company after deducting there from, accumulated balance of loss, deferred revenue expenditure, and other intangible assets; and (b) further reduced by the amounts representing investments of such company in shares of its subsidiaries, companies in the same group, all other NBFCs, and the book value of debentures, bonds, outstanding loans & advances (including hire purchase and lease finance) made to, and deposits with subsidiaries of such company and companies in the same group, to the extent such book value exceeds 10% of (a) above."
  },
  {
    "question": "What are the responsibilities of the NBFCs registered with Reserve Bank, with regard to submission on compliances and other information?",
    "answer": "NBFCs shall comply with the provisions of the Master Direction – Reserve Bank of India (Filing of Supervisory Returns) Directions – 2024 (as amended from time to time) for submission of various supervisory returns to the Reserve Bank."
  },
  {
    "question": "What are the regulations for lending against security of shares by NBFCs?",
    "answer": "NBFCs shall comply with the regulations contained in para 36 of the Master Direction – Reserve Bank of India (Non-Banking Financial Company – Scale Based Regulation) 2023 (as amended from time to time) while granting loans against security of shares. The regulations include, inter alia, maintaining a Loan to Value (LTV) ratio of 50% at all times, accept only Group 1 securities as collateral for loans of value more than ₹5 lakh where lending is done for investment in capital markets, undertake necessary reporting to stock exchanges on shares pledged in their favour, etc. In addition to the above, there are other related regulations on NBFCs viz., there shall be ceiling of ₹1 crore per borrower for financing subscription to Initial Public Offer (IPO) and NBFCs can fix more conservative limits. Further, NBFCs are prohibited from lending against security of their own shares and debentures."
  },

  {
    "question": "Will the regulations on Lending against security of shares be applicable on restructured accounts?",
    "answer": "The resolution of stressed assets are subject to the provisions of (a) the Prudential Framework for Resolution of Stressed Assets as contained in para 18 and (b) norms on restructuring of advances as contained in para 22, 23, 24 and 25 of the Master Direction – Reserve Bank of India (Non-Banking Financial Company – Scale Based Regulation) 2023 (as amended from time to time). The acquisition of shares due to conversion of debt into equity during a restructuring process will be exempted from regulatory ceilings on capital market exposures."
  },
  {
    "question": "Will the regulations on Lending against security of shares be applicable on those loans where the primary security is not shares/ units of mutual funds?",
    "answer": "Loans which are against the collateral of multiple securities and it is specifically agreed to in the agreement that primary security would be something other than shares/ units of mutual funds, LTV as would not be applicable. However, reporting requirements shall remain. In cases where such differentiation is not made (thereby NBFCs can off-load shares at the instance of a default), LTV would be applicable."
  },
  {
    "question": "Whether LTV for loans against security of shares is to be computed at scrip level or at portfolio level?",
    "answer": "LTV would be computed at portfolio level."
  },
  {
    "question": "Whether PoA/ Non-Disposal undertaking structure by whatever name called is covered under the regulatory prescriptions on Lending against security of shares?",
    "answer": "The regulations would be applicable and the type of encumbrance created is immaterial."
  },
  {
    "question": "Does the definition of “companies in a group” as given in the Master Direction – Reserve Bank of India (Non-Banking Financial Company – Scale Based Regulation) 2023 apply in respect of concentration of credit/ investment norms.",
    "answer": "No, the definition of “companies in the group” is only for the purpose of determining the applicability of prudential norms on multiple NBFCs in a group."
  },
  {
    "question": "Whether acquisition/ transfer of shareholding of 26 per cent or more of the paid-up equity capital of an NBFC within the same group i.e., intra group transfers require prior approval of the Bank?",
    "answer": "Yes, prior approval would be required in all cases of acquisition/ transfer of shareholding of 26 per cent or more of the paid up equity capital of an NBFC."
  },
  {
    "question": "NBFCs are charging high interest rates from their borrowers. Is there any ceiling on interest rate charged by the NBFCs to their borrowers?",
    "answer": "Reserve Bank of India has deregulated interest rates to be charged to borrowers by NBFCs. The rate of interest to be charged by the company is governed by the terms and conditions of the loan agreement entered into between the borrower and the NBFCs. However, the NBFCs have to be transparent and the rate of interest and manner of arriving at the rate of interest to different categories of borrowers should be disclosed to the borrower or customer in the application form and communicated explicitly in the sanction letter and on their websites, Key Facts Statement, etc., to enable the borrower to take an informed decision."
  },
  {
    "question": "RBI permits NBFCs to hedge their exposure through dealing in IRFs. Whether IRFs can be used holistically for hedging assets and liabilities in dynamic interest rate scenarios within total Balance Sheet amount and within hedging definition?",
    "answer": "IRF may be used to hedge interest rate risk associated with single asset/ liability or a group of assets/ liabilities. Hence, NBFCs are permitted to use duration-based hedging for managing interest rate risk."
  },
  {
    "question": "Whether NBFCs as trading member can participate in the IRF market only for hedging or can also take trading position?",
    "answer": "As per extant guidelines, NBFCs with asset size of ₹1,000 crore and above are permitted to participate in IRF as trading members duly subject to provisions of ‘Rupee Interest Rate Derivatives (Reserve Bank) Directions, 2019’ dated June 26, 2019 (as amended from time to time). While trading members of stock exchanges are permitted to execute trades on their own account as well as on account of their clients, only banks, SPDs and All India Financial Institutions (AIFIs) have been allowed to act as market-makers. Hence, currently, NBFCs as trading members are permitted to execute only their proprietary trades and are not allowed to undertake transactions on behalf of clients."
  },
  {
  "question": "What is a Residuary Non-Banking Company (RNBC)? In what way is it different from other NBFCs?",
  "answer": "Residuary Non-Banking Company is a class of NBFC which is a company and has as its principal business the receiving of deposits, under any scheme or arrangement or in any other manner and not being an Investment and Credit Company, a housing finance company, an insurance company, a factor, a mutual benefit company, a mutual benefit financial company and a miscellaneous non-banking company. These companies are required to maintain investments as per directions of the Reserve Bank, in addition to liquid assets. The functioning of these companies is different from those of NBFCs in terms of method of mobilization of deposits and requirement of deployment of depositors' funds as per Directions. Besides, Prudential Norms Directions are also applicable to these companies."
},

{
  "question": "We understand that there is no ceiling on raising of deposits by RNBCs, then how safe is deposit with them?",
  "answer": "While there is no ceiling on raising of deposits by RNBCs, it is mandated that every RNBC has to ensure that the amounts deposited with it are fully invested in approved investments. In order to secure the interests of depositors, such companies are required to invest 100 per cent of their deposit liability into highly liquid and secure instruments, namely, Central/State Government securities, fixed deposits with scheduled commercial banks (SCB), Certificate of Deposits of SCB/FIs, units of Mutual Funds, etc."
},

{
  "question": "Can RNBC forfeit deposit if deposit instalments are not paid regularly or discontinued?",
  "answer": "No. Residuary Non-Banking Company cannot forfeit any amount deposited by the depositor, or any interest, premium, bonus or other advantage accrued thereon."
},

{
  "question": "What is the rate of interest that an RNBC must pay on deposits and what should be maturity period of deposits taken by them?",
  "answer": "The minimum interest an RNBC should pay on deposits should be 5% (to be compounded annually) on the amount deposited in lump sum or at monthly or longer intervals; and 3.5% (to be compounded annually) on the amount deposited under daily deposit scheme. Interest includes premium, bonus or any other advantage, that an RNBC promises to the depositor by way of return. An RNBC can accept deposits for a minimum period of 12 months and maximum period of 84 months from the date of receipt of such deposit. They cannot accept deposits repayable on demand."
},

{
  "question": "What is ‘deposit’ and ‘public deposit’? Is it defined anywhere?",
  "answer": "The term ‘deposit’ is defined under Section 45 I(bb) of the RBI Act, 1934. ‘Deposit’ includes and shall be deemed always to have included any receipt of money by way of deposit or loan or in any other form but does not include: (i) amount raised by way of share capital, or contributed as capital by partners of a firm; (ii) amount received from a scheduled bank, a co-operative bank, a banking company, Development bank, State Financial Corporation, IDBI or any other institution specified by RBI; (iii) amount received in ordinary course of business by way of security deposit, dealership deposit, earnest money, advance against orders for goods, properties or services; (iv) amount received by a registered money lender other than a body corporate; (v) amount received by way of subscriptions in respect of a ‘Chit’. Para 3(xiii) of the Master Direction – Non-Banking Financial Companies Acceptance of Public Deposits (Reserve Bank) Directions, 2016 defines a ‘public deposit’ as a ‘deposit’ as defined under Section 45I(bb) of the RBI Act, 1934 and further excludes several categories such as amounts received from government, financial institutions, other companies, directors/shareholders (with conditions), debentures, infrastructure bonds, etc. Thus, the directions exclude from the definition of public deposit, amounts raised from certain set of informed lenders who can make independent decision."
},

{
  "question": "Which entities can legally accept deposits from public?",
  "answer": "Banks, including co-operative banks, can accept deposits. NBFCs (including Housing Finance Companies) which have been issued Certificate of Registration by the Reserve Bank with a specific licence to accept deposits, are entitled to accept public deposits. Only those NBFCs holding a deposit accepting Certificate of Registration can accept deposits, and only to the permissible extent. Companies authorized by the Ministry of Corporate Affairs under the Companies (Acceptance of Deposits) Rules can also accept deposits up to a certain limit. Cooperative Credit Societies can accept deposits from their members but not from the general public. It is not legally permissible for other entities to accept public deposits. Unincorporated bodies like individuals, partnership firms, and associations of individuals are prohibited from accepting deposits. Further, the First Schedule of the ‘Banning of Unregulated Deposit Schemes Act, 2019’ lists regulated deposit schemes."
},
{
  "question": "Can all NBFCs accept deposits? Is there any ceiling on acceptance of Public Deposits? What is the rate of interest and period of deposit which NBFCs can accept?",
  "answer": "All NBFCs are not entitled to accept public deposits. Only those NBFCs that hold a deposit accepting Certificate of Registration, and have a minimum investment grade credit rating of ‘BBB–‘ from any of the SEBI-registered credit rating agencies, are allowed to accept/ hold public deposits up to a limit of 1.5 times of their Net Owned Funds. Presently, the maximum rate of interest an NBFC can offer is 12.5%. The interest may be paid or compounded at rests not shorter than monthly rests. The NBFCs are allowed to accept/ renew public deposits which are repayable after 12 months but not later than 60 months. They cannot accept deposits repayable on demand. However, as a matter of public policy, Reserve Bank has decided that only banks should be allowed to accept public deposits and as such, has, since 1997, not issued any Certificate of Registration (CoR) to new NBFCs for acceptance of public deposits."
},

{
  "question": "In respect of companies which do not fulfill the 50-50 criteria but are accepting deposits – do they come under Reserve Bank’s purview?",
  "answer": "A company which does not have financial assets which are more than 50% of its total assets and does not derive at least 50% of its gross income from such assets is not an NBFC. Its principal business would be non-financial activity like agricultural operations, industrial activity, purchase or sale of goods or purchase/construction of immovable property, and will be a non-banking non-financial company. Acceptance of deposits by a Non-Banking Non-Financial Company are governed by the rules and regulations issued by the Ministry of Corporate Affairs."
},

{
  "question": "Why is the Reserve Bank so restrictive in allowing NBFCs to raise public deposits?",
  "answer": "The Reserve Bank's overarching concern while regulating/ supervising any financial entity is protection of depositors' interest. Depositors place deposit with any entity on trust unlike an investor who invests in the shares of a company with the intention of sharing the risk as well as return with the promoters. Protection of depositors' interest, thus, is supreme in financial regulation. Further, the deposits of NBFCs do not have insurance from the Deposit Insurance and Credit Guarantee Corporation."
},

{
  "question": "Which are the NBFCs specifically authorized by the Reserve Bank to accept deposits?",
  "answer": "The Reserve Bank publishes the list of NBFCs that hold a valid Certificate of Registration for accepting deposits on its website (www.rbi.org.in) under Regulation → Non-Banking → NBFCs. Members of the public are advised to check the list before placing deposits with NBFCs."
},

{
  "question": "Whether NBFCs can accept deposits from NRIs?",
  "answer": "NBFCs can accept deposits from NRIs subject to compliance with Foreign Exchange Management (Deposit) Regulations 2016 (as amended from time to time) and also subject to the condition that the rate of interest on these deposits shall not exceed the rate specified by the Reserve Bank for such deposits with scheduled commercial banks."
},

{
  "question": "Can a Co-operative Credit Society accept deposits from the public?",
  "answer": "No. Co-operative Credit Societies cannot accept deposits from general public. They can accept deposits only from their members within the limit specified in their bye laws. It may be noted that Co-operative Credit Societies do not fall under the regulatory purview of the Reserve Bank and fall under the purview of Registrar of Cooperative Societies."
},

{
  "question": "Can a Salary Earners’ Society accept deposits from the public?",
  "answer": "No. These societies are formed for salaried employees and hence they can accept deposit only from their own members and not from general public. It may be noted that Salary Earners’ Societies do not fall under the regulatory purview of the Reserve Bank and fall under the purview of Registrar of Cooperative Societies."
},

{
  "question": "Is nomination facility available to the Depositors of NBFCs?",
  "answer": "Yes, nomination facility is available to the depositors of NBFCs. The Rules for nomination facility are provided for in Section 45QB of the RBI Act, 1934. NBFCs have been advised to adopt the Banking Companies (Nomination) Rules, 1985 made under Section 45ZA of the Banking Regulation Act, 1949. Accordingly, depositor/s of NBFCs are permitted to nominate a person to whom the NBFC can return the deposit in the event of the death of the depositor/s. NBFCs are advised to accept nominations made by the depositors in the form similar to one specified under the said rules, viz., Form DA 1 for the purpose of nomination, and Form DA2 and DA3 for cancellation of nomination and change of nomination respectively."
},

{
  "question": "How does the Reserve Bank come to know about unauthorized acceptance of deposits by companies not registered with it or of NBFCs engaged in lending or investment activities without obtaining the Certificate of Registration from it?",
  "answer": "NBFCs that ought to have sought registration from the Reserve Bank but are functioning without doing so are committing a breach of law. Such companies are liable for action as envisaged under the RBI Act, 1934. To identify such entities, the Reserve Bank has multiple sources of information. These include market intelligence, complaints received from affected parties, industry sources, and exception reports submitted by statutory auditors in terms of Master Direction - Non-Banking Financial Companies Auditor’s Report (Reserve Bank) Directions, 2016 (as amended from time to time). Further, the State Level Co-ordination Committees (SLCC) is convened by the Reserve Bank in all the States/UTs on quarterly basis. The SLCC is chaired by the Chief Secretary/ Administrator of the concerned State/UT and has, as its members, apart from the Reserve Bank, the Regional Directorate of the MCA/ ROC, local unit of SEBI, NHB, Registrar of Chits, ICAI, Economic Intelligence Unit of the State Police and officials from Law and Home Ministries of the State Government. As all the relevant financial sector regulators and enforcement agencies participate in the SLCC, it is possible to quickly share the information and agree on an effective course of action to be taken against entities indulging in unauthorized and suspect businesses involving funds mobilization from public."
},

{
  "question": "Can Proprietorship/Partnership Concerns associated/not associated with registered NBFCs accept public deposits? What action can be taken if such unincorporated entities accept public deposits?",
  "answer": "No. Proprietorship and partnership concerns are un-incorporated bodies. Hence, they are prohibited under the RBI Act 1934 from accepting public deposits. Such unincorporated entities, if found accepting public deposits, are liable for penal action under the Act."
},

{
  "question": "There are many jewellery shops taking money from the public in instalments. Is this amounting to acceptance of deposits?",
  "answer": "It depends on whether the money is received as advance for delivering jewellery at a future date or whether the money is received with a promise to return the same with interest. The money accepted by Jewellery shops in instalments for the purpose of delivering jewellery at the end of the period of contract is not deposit. It will amount to acceptance of deposits if in return for the money received, the jewellery shop promises to return the principal amount along with interest."
},

{
  "question": "What if NBFCs which have not been authorized to accept public deposits use proprietorship/partnership firms floated by their promoters to collect deposits?",
  "answer": "NBFCs are prohibited by the Reserve Bank from associating with any unincorporated bodies. If NBFCs associate themselves with unincorporated entities which are accepting deposits in contravention of RBI Act, they are also liable for penal action under the Act, or action under the Protection of Interest of Depositors (in Financial Establishments) Act, or the Banning of Unregulated Deposit Schemes Act, 2019, as applicable."
},

{
  "question": "What is the difference between acceptance of money by Chit Funds and acceptance of deposits?",
  "answer": "Deposits are defined under the RBI Act 1934 as acceptance of money other than that raised by way of share capital, money received from banks and other financial institutions, money received as security deposit, earnest money and advance against goods or services and subscriptions to chits. All other amounts received in any form are treated as deposits. Chit Funds activity involves contributions by members in instalments by way of subscription to the Chit and by rotation each member of the Chit receives the chit amount. The subscriptions are specifically excluded from the definition of deposits and cannot be termed as deposits. While Chit funds may collect subscriptions as above, they are prohibited by the Reserve Bank from accepting deposits from public (except from shareholders) with effect from August 2009."
},

  {
    "question": "What are the salient features of NBFC regulations which the depositor may note at the time of investment?",
    "answer": "NBFCs are allowed to accept/renew public deposits only for 12 to 60 months and cannot accept deposits repayable on demand. They cannot offer interest rates higher than the RBI ceiling, currently 12.5% per annum, with rests not shorter than monthly. They must maintain a minimum investment grade credit rating of ‘BBB–’ or above, reviewed annually. Deposits are not insured, and RBI does not guarantee repayment. Certain mandatory disclosures must be made in the deposit application form."
  },
  {
    "question": "What precautions should a depositor take before placing deposit with an NBFC?",
    "answer": "Depositors should check that the NBFC is registered with RBI and authorized to accept deposits. They must verify the Certificate of Registration displayed at the office. The interest rate offered should not exceed 12.5%. Proper receipts must be obtained with all details of deposit. Brokers/agents must be confirmed as authorized. Depositors should remember that NBFC deposits are unsecured and not insured. RBI does not guarantee repayment or financial soundness of NBFCs."
  },
  {
    "question": "Does the Reserve Bank guarantee the repayment of the deposits collected by NBFCs?",
    "answer": "No. RBI does not guarantee repayment of deposits collected by NBFCs even though they may be authorized to accept deposits."
  },
  {
    "question": "In case an NBFC defaults in repayment of deposit, what course of action can be taken by depositors?",
    "answer": "Depositors can approach the National Company Law Tribunal (NCLT), earlier CLB, Consumer Forum, or file a civil suit. State Governments under their depositor protection laws can also act, including attaching properties in fraudulent cases. NBFCs are required to have internal grievance redressal mechanisms as well."
  },
  {
    "question": "What is the role of National Company Law Tribunal (NCLT) in protecting the interest of depositors? How can one approach it?",
    "answer": "If an NBFC fails to repay deposits, NCLT can order repayment within a specified time and conditions. Depositors can approach NCLT by submitting an application in the prescribed form to the appropriate bench based on jurisdiction."
  },
  {
    "question": "Can you give the addresses of the various benches of the National Company Law Tribunal (NCLT), indicating their respective jurisdiction?",
    "answer": "The details of addresses and territorial jurisdiction of the NCLT benches are available on the official website https://nclt.gov.in/about-nclt."
  },
  {
    "question": "What is the procedure adopted by the Official Liquidator when appointed on defaulting NBFCs?",
    "answer": "An Official Liquidator is appointed by the court after hearing the company in a winding-up petition. The liquidator takes custody of the company’s property, prepares a statement of affairs, and submits a repayment scheme to the court. Assets are realized and distributed to creditors as per court-approved scheme. Claims are invited from depositors via public notices. For NBFCs with assets of ₹500 crore or more, resolution may proceed under the Insolvency and Bankruptcy Code, 2016."
  },
  {
    "question": "Can one approach Consumer Forum, Civil Court, and NCLT simultaneously for deposit-related grievances?",
    "answer": "Yes. The depositor is entitled to approach the relevant authorities as permissible under law."
  },
  {
    "question": "Is there an Ombudsman for hearing complaints against NBFCs or does the Reserve Bank have any grievance redressal mechanism in place for NBFCs?",
    "answer": "NBFCs are required to set up internal grievance redressal mechanisms. Additionally, deposit-taking NBFCs or those with customer interface and asset size of ₹100 crore or more are covered under the RBI Integrated Ombudsman Scheme, 2021. If complaints remain unresolved for more than one month, customers may approach the Ombudsman via the CMS portal at https://cms.rbi.org.in."
  },
  {
    "question": "What is the recourse available to depositors if companies registered with MCA but not with RBI default in repayment?",
    "answer": "Such companies are not under RBI’s regulation. Complaints should be lodged with the Registrar of Companies (RoC) of the respective state. RBI may take action if such companies were required to be registered as NBFCs but failed to do so and accepted deposits illegally."
  },
  {
    "question": "Are NBFCs liable to pay interest on overdue matured deposits?",
    "answer": "Yes. NBFCs must pay overdue interest from the date of claim or maturity, whichever is later, until repayment. If the claim is lodged after maturity, interest is payable from the date of claim until repayment, while payment for the period between maturity and claim is at the company’s discretion. In cases where deposits are frozen due to government orders, renewal or repayment must follow specific procedures, including renewal advice and proper notification to authorities."
  },
{
  "question": "Can an NBFC pre-pay its public deposits?",
  "answer": "An NBFC accepts deposits under a mutual contract with its depositors. In case a depositor requests for pre-mature payment, the Reserve Bank has prescribed regulations for such an eventuality in the Master Direction - Non-Banking Financial Companies Acceptance of Public Deposits (Reserve Bank) Directions, 2016 (as amended from time to time) wherein it is specified that NBFCs cannot grant any loan against a public deposit or make premature repayment of a public deposit within a period of three months (lock-in period) from the date of its acceptance. However, in the event of death of a depositor, the NBFC may, even within the lock-in period, repay the deposit at the request of the joint holders with survivor clause, or to the nominee / legal heir only against submission of relevant proof, to the satisfaction of the NBFC. Further, in order to enable a depositor to meet the expenses of an emergent nature, the NBFC may subject to satisfaction of the NBFC about the circumstances, prematurely repay tiny deposits (i.e., up to ₹10,000/-) and also other deposits, as per the provisions laid down by the Reserve Bank.\n\nAn NBFC, which is not a problem company, subject to above provisions, may permit premature repayment of a public deposit after the lock–in period, at its sole discretion, at the rate of interest prescribed by the Reserve Bank.\n\nA problem NBFC is prohibited from making premature repayment of any deposits or granting any loan against public deposits, as the case may be. The prohibition shall not, however, apply in the case of death of depositor or repayment of tiny deposits (i.e. up to ₹10,000/-) in order to enable a depositor to meet expenses of an emergent nature, subject to lock in period of 3 months in the latter case."
},
{
  "question": " What is the liquid assets requirement for the deposit taking companies? Where are these assets kept? Do depositors have any claims on them?",
  "answer": "In terms of Section 45-IB of the RBI Act, 1934, the minimum level of liquid assets to be maintained by deposit taking NBFCs is 15% of public deposits outstanding as on the last working day of the second preceding quarter. Of the 15%, NBFCs are required to invest not less than 10% in approved securities and the remaining 5% can be in unencumbered term deposits with any Scheduled Commercial Bank, SIDBI, or NABARD, or bonds issued by SIDBI or NABARD. The liquid assets may consist of Government securities, Government guaranteed bonds, and term deposits/bonds. Investment in Government Securities should be in dematerialised form maintained in Subsidiary General Ledger Account with RBI or Gilt Account with CSGL, or a dematerialised account with a depository participant registered with SEBI. Government guaranteed bonds may be kept in dematerialised form with SCB/SHCIL or in a dematerialised account with a depository. Physical government bonds may be kept in safe custody of SCB/SHCIL along with term deposits. NBFCs must maintain liquid assets at their registered office or elsewhere with RBI permission. Deposits are unsecured and depositors do not have direct claim on these liquid assets."
},
{
"question": "Who rates deposit taking NBFCs for acceptance of deposit?",
"answer": "NBFCs shall obtain credit rating from any of the SEBI-registered credit rating agencies."
},
{
"question": "What is the minimum investment grade rating for deposit taking NBFCs? When an NBFC’s rating is downgraded, does it have to bring down its level of public deposits immediately or over a period of time?",
"answer": "The minimum investment grade credit rating for deposit taking NBFCs is ‘BBB–‘. If the credit rating is downgraded below this level, the NBFC must stop accepting public deposits and stop renewing existing deposits immediately. However, existing deposits are allowed to run off to maturity. The NBFC must also report the position to the Reserve Bank within fifteen working days."
},
{
"question": "What is the purpose of enacting Protection of Interest of Depositors in Financial Establishments Act by the State Governments?",
"answer": "The purpose of the Act is to protect the interests of depositors. RBI regulations ensure NBFCs function on sound lines but do not have specific provisions to recover defaulting assets. The State Governments, through this Act, are empowered to attach and sell assets of defaulting companies, entities, or officials to protect depositors."
},
{
"question": " Will the passage of the Protection of Interest of Depositors in Financial Establishments by the State Governments help in nailing unincorporated entities and companies from unauthorizedly accepting deposits?",
"answer": "Yes. The Act makes unauthorized acceptance of deposits a cognizable offence, allowing imprisonment and prosecution. State Governments can attach properties of such entities and distribute proceeds to depositors. The Banning of Unregulated Deposit Schemes Act, 2019, further strengthens mechanisms to prevent unregulated deposit schemes, including attachment and sale of property, designated courts, powers for investigation, search & seizure, and penal provisions."
},
{
"question": "Still there are cases of unscrupulous financial entities cheating public time and again. How does the Reserve Bank plan to strengthen its surveillance on unauthorized acceptance of deposits/unauthorized conduct of NBFI business by companies?",
"answer": "The Reserve Bank is strengthening its market intelligence function and monitoring financials of companies through complaints and intelligence. The Sachet portal allows the public to report unauthorized deposit collections. RBI also emphasizes public awareness that high returns involve high risks, and investments should be made only with regulated entities."
},
{
"question": "Are Collective Investment Schemes (CIS) regulated by the Reserve Bank?",
"answer": "No. CIS do not fall under the regulatory purview of the Reserve Bank and are regulated by SEBI."
},
{
"question": "Which is the authority that regulates Collective Investment Schemes (CIS)?",
"answer": "SEBI regulates CIS. Complaints or grievances about CIS promoters should be forwarded to SEBI and/or the Economic Offences Wing/Police Department of the State Government."
},
{
"question": "Is the conducting of Chit Fund business permissible under law?",
"answer": "Yes. Chit funds are governed by the Chit Funds Act, 1982, a Central Act administered by State Governments. Only chit funds registered under this Act can legally operate."
},
{
"question": " If Chit Fund companies are financial entities, why are they not regulated by the Reserve Bank?",
"answer": "Chit Fund companies are regulated under the Chit Funds Act, 1982 by State Governments. To avoid dual regulation, chit fund companies are exempted from provisions of Sections 45-IA, 45-IB, and 45-IC of the RBI Act, 1934. RBI prohibits chit funds from accepting public deposits since 2009 and can prosecute them if they do."
},
{
"question": "There are some companies like Multi-Level Marketing companies, Direct Selling Companies, Online Selling Companies. Do they come under the purview of the Reserve Bank?",
"answer": "No. These companies do not fall under RBI purview. They are regulated by respective State Governments and the provisions of the Consumer Protection Act, 2019 and Consumer Protection (Direct Selling) Rules, 2021."
},
{
"question": "What are money circulation/Ponzi/multi-level marketing schemes?",
"answer": "These schemes promise easy or quick money upon enrolling members. Multi-level marketing or pyramid schemes derive income mostly from enrolling new members and collecting subscription fees, rather than from selling products. Ponzi schemes pay returns to existing members from funds collected from new members, creating no real asset growth. These schemes collapse when the inflow of new funds stops, often resulting in losses for members at the bottom."
},
{
"question": "Is acceptance of money under Money Circulation/Multi-level Marketing/Pyramid structured schemes allowed? Does the Reserve Bank regulate such schemes?",
"answer": "No. Acceptance of money under such schemes is illegal and a cognizable offence under the Prize Chit and Money Circulation (Banning) Act, 1978. RBI does not regulate these schemes directly but assists the Central Government in framing rules under the Act."
},
{
"question": "Then who regulates entities that run such schemes?",
"answer": "State Governments regulate and monitor Money Circulation, Ponzi, and MLM schemes under the Prize Chits and Money Circulation Schemes (Banning) Act, 1978. Contraventions are handled by State authorities."
},
{
"question": "What if someone operates such a scheme?",
"answer": "Complaints or information about such schemes should be reported to the police/Economic Offences Wing (EOW) of the concerned State Government or the Ministry of Corporate Affairs. RBI forwards such complaints to the relevant State authorities."
},
{
"question": "What are Unincorporated Bodies (UIBs)? Does the Reserve Bank have any role to play in curbing illegal deposit acceptance activities of UIBs? Who has the power to take action against UIBs accepting deposits?",
"answer": "UIBs include individuals, firms, or unincorporated associations. They are prohibited from accepting deposits under Section 45S of the RBI Act, 1934. Violations can lead to imprisonment or fines. State Governments take action against UIBs, while RBI forwards complaints to State EOW. Both RBI and State Governments can apply for search warrants under Section 45T to take action against offenders. Public awareness campaigns by RBI also help curb illegal activities of UIBs."
},
{
"question": "There are some entities (not companies) which carry on activities like that of NBFCs. Are they allowed to take deposits? Who regulates them?",
"answer": "Individuals, firms, or unincorporated associations engaging in lending, investment, hire-purchase, or leasing cannot accept deposits except by way of loan from relatives. They are not allowed to take public deposits."
},
{
"question": "What precautions have to be taken by the public to forewarn themselves about the likelihood of losing money in schemes that offer high rates of interest?",
"answer": "Before investing in high-return schemes, depositors/investors must ensure the entity is registered with a Financial Sector Regulator and authorized to accept funds. High promised returns usually involve higher risks. Depositors should be aware that speculative investments can result in loss of principal."
},
{
"question": "Who can the Depositor/Investor turn to in case of grievances?",
"answer": "Depositors/investors can refer to the regulators listed in the First Schedule of the Banning of Unregulated Deposit Schemes Act, 2019. For banned activities, grievances may be lodged with the State Police or Economic Offences Wing of the State Police."
},
{
"question": "What constitutes Commercial Real Estate exposure?",
"answer": "Commercial Real Estate (CRE) consists of loans to builders, developers, or others for creation/acquisition of commercial real estate such as office buildings, retail spaces, multi-purpose commercial premises, industrial or warehouse space, hotels, land acquisition, development and construction, etc., where repayment depends on cash flows from lease/rental payments or sale. Loans for third dwelling unit onwards to an individual are also treated as CRE exposure. Non-fund based limits are included. CRE–Residential Housing (CRE–RH) is a sub-category for residential housing projects, with integrated commercial space allowed up to 10% of total Floor Space Index (FSI). Beyond 10%, the loan is classified as CRE and not CRE–RH."
},
{
"question": "In terms of the circular ‘Multiple NBFCs in a Group: Classification in Middle Layer’ dated October 11, 2022, total assets of all NBFCs in a group shall be consolidated to determine the threshold for their classification in the Middle Layer under SBR Framework for NBFCs. Will this consolidation of assets apply to unregistered Core Investment Companies (CICs) in the group?",
"answer": "No, only NBFCs with a Certificate of Registration from RBI are consolidated for the Middle Layer classification. However, if unregistered CICs with assets below ₹100 crore have accessed public funds, their assets are consolidated for this purpose, without changing their unregistered status."
},
{
"question": "Whether LTV of 50% will also apply to lending against units of mutual funds?",
"answer": "Loans against units of mutual funds, except exclusively debt-oriented funds, will attract LTV requirements similar to loans against shares. LTV for loans against exclusively debt-oriented mutual funds may be decided by individual NBFCs as per their loan policy."
},
{
"question": "Is prior written approval of the Reserve Bank required in cases of merger of an NBFC ‘A’, with another NBFC/ entity ‘B’?",
"answer": "Yes, prior written approval of RBI is required for NBFC ‘A’. If NBFC ‘A’ ceases to exist post-merger, its Certificate of Registration must be surrendered. If ‘B’ is an NBFC, prior RBI approval is required for change in control, change in shareholding by 26% or more, or change in management affecting more than 30% of directors (excluding independent directors). If ‘B’ is not an NBFC but meets Principal Business Criteria (50-50) post-merger, it must also seek RBI approval and registration as NBFC."
},
{
"question": "Is prior written approval of the Reserve Bank required in cases of merger of an entity (not an NBFC) with an NBFC?",
"answer": "Yes, prior RBI approval is required if the merger results in: (i) change in control of the NBFC, (ii) change in shareholding of 26% or more of paid-up equity capital, or (iii) change in management affecting more than 30% of directors excluding independent directors. The NBFC must continue to meet the Principal Business Criteria (50-50) post-merger to retain its Certificate of Registration."
},
{
"question": "Is prior written approval of the Reserve Bank required in cases of amalgamation of an NBFC ‘A’, with another NBFC/ entity ‘B’?",
"answer": "Yes, NBFCs being amalgamated must obtain prior written approval from RBI. Requisite approvals under regulations must be sought depending on the nature of the amalgamation or merger proposal."
},
{
"question": "Is prior written approval of the Reserve Bank required before approaching any Court or Tribunal for seeking orders for merger/ amalgamation involving NBFCs?",
"answer": "Yes, prior RBI approval must be obtained before approaching any Court or Tribunal for merger or amalgamation orders involving NBFCs, including cases covered under FAQs 84, 85, and 86."
}
             ]


In [ ]:
from langchain.evaluation import load_evaluator

qa_evaluator = load_evaluator("qa", llm=llm)


for faq in rbi_faqs:

    model_answer = ask_from_pdf(faq["question"])


    evaluation = qa_evaluator.evaluate_strings(
        prediction=[model_answer],
        reference=[faq["answer"]],
        input=[faq["question"]]  )

    print("Question:", faq["question"])
    print("Prediction:", model_answer)
    print("Evaluation:", evaluation)
    if evaluation.get('score', 0) == 1:
        break
    else:
        continue



Question: What is a Non-Banking Financial Company (NBFC)?
Prediction: Based on the provided text, a specific type of Non-Banking Financial Company, an **NBFC-ICC (Investment and Credit Company)**, is defined as:

A company that is a financial institution whose principal business is asset finance, providing finance (whether by making loans, advances, or otherwise) for any activity other than its own, and the acquisition of securities.

The text also mentions that an **NBFC-Factor** is defined in clause (f) of section 45-I of the RBI Act, 1934.
Evaluation: {'reasoning': 'INCORRECT', 'value': 'INCORRECT', 'score': 0}


* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 2
Please retry in 42.053455556s. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 2
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 42
}
].
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 2
Please retry in 39.947150545s. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-pro"
  }


Question: What does conducting financial activity as “principal business” mean?
Prediction: Based on the provided text, conducting financial activity as a "principal business" means a company meets a two-part test known as the Principal Business Criteria.

A company is considered to have financial activity as its principal business if it satisfies **both** of the following conditions:

1.  **Asset Test:** Its financial assets are more than 50% of its total assets (after subtracting intangible assets).
2.  **Income Test:** Its income from financial assets is more than 50% of its gross income.

Both of these tests, based on the company's last audited balance sheet, must be met for it to be treated as a Non-Banking Financial Company (NBFC).
Evaluation: {'reasoning': 'GRADE: CORRECT', 'value': 'CORRECT', 'score': 1}
